In [ ]:
import os
from glob import glob

import pairwise_fte
from all_optimizations import ekf
from lib.app import create_labeled_videos

%load_ext autoreload
%autoreload 2

%matplotlib inline

root_dir = os.path.join("..", "data")
# If you are running the code remotely on the Linux i9.
# root_dir = os.path.join("/","data", "dlc", "to_analyse", "cheetah_videos")

In [ ]:
# Generate the pose functions if this has not been done before (note this should only be performed once). It will be saved in the root_dir.
functions = pairwise_fte.create_pose_functions(root_dir)

In [ ]:
# Configurations for the optimisation.
data_path = os.path.join("2017_09_02", "bottom", "jules", "run2")
start_frame = 100
end_frame = 180
dlc_thresh = 0.5

In [ ]:
# Run the optimisation
pairwise_fte.run(root_dir, data_path, start_frame, end_frame, dlc_thresh, auto_frame_select=False)

In [ ]:
# Create 2D reprojection videos.
video_fpaths = sorted(glob(os.path.join(root_dir, data_path, "cam[1-9].mp4"))) # original vids should be in the parent dir
create_labeled_videos(video_fpaths, out_dir=os.path.join(root_dir, data_path, "fte_pw"), draw_skeleton=True, pcutoff=dlc_thresh)

In [ ]:
# Compare the optimisation with previous results i.e. compare the output pickle file fte.pickle. This is a visual inspection.abs
orig_file = os.path.join(root_dir, data_path, "ekf", "ekf.pickle")
pw_file = os.path.join(root_dir, data_path, "fte_pw", "fte.pickle")
pairwise_fte.compare(orig_file, pw_file)

In [ ]:
# DEBUB purposes - if you want to display particular frames to inspect the measurements that are incorporated in the optimisation process.
# The camera to use.
cam_num = 1
# The measurements to display 1 - base prediction, 2 - 3 pairwise measurements. Max is 3.
pw_values = (1, 2, 3)
# Frame number to display.
frame_num = 110
pairwise_fte.display_test_image(data_dir, cam_num, pw_values, frame_num)